# 📘 Nuggetizer: A lightweight nugget-based evaluation framework for pyterrier-rag

## 📌 Introduction
In this notebook, we demonstrate how to evaluate a Retrieval-Augmented Generation (RAG) system
using a semantic nugget-based evaluation framework inspired by the "AutoNuggetizer" used in TREC 2024.
The goal is to assess the factual informativeness of generated answers through fine-grained nugget detection
and scoring. This setup is general and compatible with Google Colab (T4 GPU).

## 🎯 Motivation and Background
- The Problem: Traditional RAG evaluations rely on lexical overlap or ROUGE scores, which miss semantic correctness.
- The Solution: Nugget evaluation, originally proposed in TREC QA 2003, revived by AutoNuggetizer, uses semantically atomic facts (“nuggets”) to evaluate answers.
- Inspiration: This library reimplements a simplified, local version of AutoNuggetizer with modular hooks into PyTerrier and HuggingFace models.

## ⚙️ Installation and Setup

In [ ]:
!pip install git+https://github.com/MattiWe/ir_datasets.git@add-msmarco-v2.1-trec-rag

In [ ]:
!pip install git+https://github.com/terrier-org/pyterrier@hf-upload-fix

In [ ]:
!pip install -q pyterrier_t5 pyterrier_pisa

In [ ]:
!pip install -q git+https://github.com/terrierteam/pyterrier_rag.git

In [ ]:
!pip install -q --no-deps ../.

In [ ]:
import pyterrier as pt
from pyterrier_rag.backend import Backend

# Dataset

In [ ]:
import ir_datasets
dataset = ir_datasets.load('msmarco-segment-v2.1')

In [ ]:
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")

# Pipelines

In [ ]:
def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

In [ ]:
import pyterrier_alpha as pta
from pyterrier_pisa import PisaIndex
from pyterrier_t5 import MonoT5ReRanker

index = pta.Artifact.load('namawho/msmarco-segment-v2.1.pisa')
bm25_ret = index.bm25() >> pt.text.get_text(pt_dataset, "segment") >> rename_pipe

monoT5 = MonoT5ReRanker(batch_size=64, verbose=False)
monoT5_ret = bm25_ret % 10 >> monoT5

In [ ]:
hf_index = pta.Artifact.from_hf('namawho/msmarco-segment-v2.1.pisa')

# Building a baseline retrieval run to generate baseline nuggets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("namawho/trec-raggy-dev")["validation"].to_pandas()
topics_df  = dataset[["qid", "query"]]
answers_df = dataset[["qid", "query", "gold_answer"]]

In [ ]:
baseline = (monoT5_ret)(topics_df.head(10))
baseline

# Nuggetizer setup

In [ ]:
from pyterrier_rag.backend import HuggingFaceBackend

backend =  HuggingFaceBackend("hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
                                          max_new_tokens=2048,
                                          model_args={
                                              "device_map": "cuda"
                                          }
                                         )

In [ ]:
from fastchat.conversation import register_conv_template, get_conv_template, Conversation, SeparatorStyle

register_conv_template(
    Conversation(
        name="meta-llama-3.1-sp",
        system_message="",
        roles=("user", "assistant"),
        sep_style=SeparatorStyle.ADD_COLON_SINGLE,
        sep="\n",
        messages=[],
    )
)

conv_template = get_conv_template("meta-llama-3.1-sp")

In [ ]:
import pandas as pd

def save_csv(path, content):
    content.to_csv(path, index=False)

def load_csv(path):
    try:
        content = pd.read_csv(path)
        return content
    except Exception:
        return None

In [ ]:
from open_nuggetizer.nuggetizer import Nuggetizer

nuggetizer = Nuggetizer(
    backend=backend, 
    conversation_template=conv_template,
    verbose=True
)

nuggets = load_csv("nuggets.csv")
if nuggets is None:
    nuggets = nuggetizer.create(baseline)
    save_csv("nuggets.csv", nuggets)

scored_nuggets = load_csv("scored_nuggets.csv")
if scored_nuggets is None:
    scored_nuggets = nuggetizer.score(nuggets)
    save_csv("scored_nuggets.csv", scored_nuggets)

# Evaluation

In [ ]:
from pyterrier_rag.prompt import Concatenator
from pyterrier_rag.readers import Reader

reader = Reader(backend)
rag_pipeline = monoT5_ret % 3 >> Concatenator() >> reader

(monoT5_ret % 3 >> Concatenator() >> reader)(topics_df.head(1))

In [ ]:
nuggetizer.VitalScore().iter_calc(baseline, scored_nuggets)

In [ ]:
import pyterrier_rag.measures

results = pt.Experiment(
    [
        rag_pipeline
    ],
    topics_df.head(2), 
    answers_df,
    [pyterrier_rag.measures.F1, nuggetizer.VitalScore()],
    #batch_size=25,
    names=['baseline retriever'],
)